In [14]:
import Pkg
Pkg.add("Ipopt")
Pkg.add("JuMP")

   Resolving package versions...
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`


In [17]:

# Este nosso modelo matematico
using JuMP, Ipopt, Statistics, LinearAlgebra

m = Model(Ipopt.Optimizer)
apartamento = zeros(4,4)
interferencias = false                                                  # esta é matriz que define o tamanho do apartamento 
interferencias = [ [0,2] , [1,2] ]                                    # posições bidimensionais das interferencias
dispositivos = [[0,0],[0,4]]  

function interference(x_dev,y_dev,x_router,y_router)  #para saber se o dispositivo conectado esta na area de interferencia usamos a seguinte lógica:
        for interferencia in interferencias #https://math.stackexchange.com/questions/1577062/how-to-know-if-a-given-point-is-inside-a-2d-circles-segment
            print("\n\ninterferencia:", interferencia)
            oa = [interferencias[1][1] - x_router, interferencias[1][2] - y_router, 0 ]
            op = [x_dev - x_router, y_dev - y_router, 0   ]
            ab = [interferencias[2][1] - interferencias[1][1], interferencias[2][2]- interferencias[1][2], 0]
            ap = [x_dev - interferencias[1][1],  y_dev - interferencias[1][2], 0  ]
            ob = [interferencias[2][1] - x_router,interferencias[2][2] - y_router, 0 ]

            print("\n\n OA:  ", oa)
            print("\n\n OP:  ", op)
            print("\n\n AB:  ", ab)
            print("\n\n AP:  ", ap)  
            print("\n\n OB:  ", ob)  

            print("\n\n cross(oa,op):  ", cross(oa,op))
            print("\n\n cross(ab,ap):  ", cross(ab,ap))  
            print("\n\n cross(ob,op):  ", cross(ob,op)) 

            if cross(oa, op) <= 0 && cross(ab,ap) >= 0 &&  cross(ob,op) >= 0
               return true
            end
        end
    return false
end

function force(x,y,std_dev)
    forca = []
    for i in dispositivos
        print("i: ", i)
        print("\n\ni[1]: ", i[1])
        print("\n\ni[2]: ", i[2])
        distancia = sqrt((x-i[1])^2+(y-i[2])^2) # a distancia entre o roteador e os dispositivos ´pode ser calculado pela distancia euclidiana
        f_= 1-0.2*distancia                     # a forca do sinal definida pela funcao apresentada no modelo matematico
        if f_ < 0 
            f_ = 0                              # não é possivel ter sinais negativos
        else
            if interference(i[1],i[2],x,y)
                f_ = 0
            end
        end
        push!(forca,f_)
    end
    if func == 1
        return Statistics.std(forca)
    else
        return sum(forca)
    end
end                                                                    # as posições dos dispositivos no nosso sistema de coordenadas        
force(2,2,1)
register(m, :force, 2, force, autodiff=true)
@variable(m, 0 <= x <= size(apartamento)[1])                            # as posições devem ser positivas
@variable(m, 0 <= y <= size(apartamento)[2])                            # as posições devem ser positivas e os roteadores da matriz não podem ser posicionados fora dos limites do apartamento


@NLconstraint(m, force(x,y,1)<= 0.4)      
@NLobjective(m, Max, force(x,y,0))                 # queremos maximizar a soma do sinal de wifi

optimize!(m)

println("x =", JuMP.value.(x))     # imprime o resultado na tela
println("y =", JuMP.value.(y))     # imprime o resultado na tela

i: [0, 0]

i[1]: 0

i[2]: 0

interferencia:[0, 2]

 OA:  [-2, 0, 0]

 OP:  [-2, -2, 0]

 AB:  [1, 0, 0]

 AP:  [0, -2, 0]

 OB:  [-1, 0, 0]

 cross(oa,op):  [0, 0, 4]

 cross(ab,ap):  [0, 0, -2]

 cross(ob,op):  [0, 0, 2]

LoadError: MethodError: no method matching isless(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  isless(::AbstractVector, [91m::AbstractVector[39m) at abstractarray.jl:2612
[0m  isless([91m::ForwardDiff.Dual{Tx}[39m, ::Integer) where Tx at C:\Users\dieme\.julia\packages\ForwardDiff\QdStj\src\dual.jl:144
[0m  isless([91m::ForwardDiff.Dual{Tx}[39m, ::Real) where Tx at C:\Users\dieme\.julia\packages\ForwardDiff\QdStj\src\dual.jl:144
[0m  ...